In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import requests
import selenium

In [2]:
path = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(path)

WebDriverException: Message: 'C:\Program Files (x86)\chromedriver.exe' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [3]:
driver.get("https://us.trend-calendar.com/trend/2020-03-01.html")

In [4]:
trend_dates = {}
pageable = True
counter = 31
while pageable == True:
    time.sleep(0.5)
    contents = driver.find_element_by_class_name("ranking")
    contents_html = contents.get_attribute("innerHTML")
    soup = bs(contents_html)
    anchors = soup.find_all("a")
    trends = []
    header = driver.find_element_by_class_name("entry-title")
    date = header.text.split("on ",1)[1]
    for a in anchors[:3]:
        url = a["href"]
        text = a.text
        d = {"URL": url, "trends": text}
        trends.append(d)
    trend_dates[date] = trends
    try:
        submit_button = driver.find_elements_by_class_name("yesterdaytomorrow")[1]
        pageable = True
        submit_button.click()
        counter -= 1
        if counter == 0:
            pageable = False
    except:
        pageable = False

In [5]:
trend_dates

{'1st March, 2020': [{'URL': 'http://twitter.com/search?q=%23TrumpSlump%20lang:en%20since%3A2020-03-01_00:00:00_EST%20until%3A2020-03-02_00:00:00_EST&src=typd',
   'trends': '#TrumpSlump'},
  {'URL': 'http://twitter.com/search?q=%22David+Byrne%22%20lang:en%20since%3A2020-03-01_00:00:00_EST%20until%3A2020-03-02_00:00:00_EST&src=typd',
   'trends': 'David Byrne'},
  {'URL': 'http://twitter.com/search?q=%22Chicago+Freestyle%22%20lang:en%20since%3A2020-03-01_00:00:00_EST%20until%3A2020-03-02_00:00:00_EST&src=typd',
   'trends': 'Chicago Freestyle'}],
 '2nd March, 2020': [{'URL': 'http://twitter.com/search?q=%23supertuesday2020%20lang:en%20since%3A2020-03-02_00:00:00_EST%20until%3A2020-03-03_00:00:00_EST&src=typd',
   'trends': '#supertuesday2020'},
  {'URL': 'http://twitter.com/search?q=%22Artemis+Fowl%22%20lang:en%20since%3A2020-03-02_00:00:00_EST%20until%3A2020-03-03_00:00:00_EST&src=typd',
   'trends': 'Artemis Fowl'},
  {'URL': 'http://twitter.com/search?q=%23CoronaOutbreak%20lang:en%2

In [ ]:
driver.quit()

In [6]:
driver.get("https://www.twitter.com/login")
time.sleep(1)
usernameInput = driver.find_element_by_name("session[username_or_email]")
passwordInput = driver.find_element_by_name("session[password]")
usernameInput.send_keys("chuck45630256")
passwordInput.send_keys("Longbow1@")
passwordInput.send_keys(Keys.ENTER)
time.sleep(5)

In [7]:
March = []
for date in trend_dates:
    for trend in trend_dates[date]:
        driver.get(trend['URL'])
        time.sleep(1.5)
        counter = 3
        text_tweets = []
        while counter != 0:
            try:
                time.sleep(2)
                contents = driver.find_element_by_class_name("css-1dbjc4n")
                contents_html = contents.get_attribute("innerHTML")
                soup = bs(contents_html)
                tweet_html = soup.find_all("div", {"data-testid":"tweet"})
                for tweet in tweet_html:
                    text = tweet.find("div", class_ = "css-901oao r-jwli3a r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0").get_text()
                    text_tweets.append(text)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(4)
                counter -= 1
            except:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(4)
                counter -= 1
        March.append(text_tweets)
        time.sleep(1)

In [ ]:
file1 = open("March tweets text.txt",encoding="utf8")

In [ ]:
data = file1.readlines()

In [ ]:
len(data)

In [ ]:
chunks = [data[x:x+5] for x in range(0, len(data), 5)]

In [ ]:
type(chunks)

In [8]:
with open('March tweets text.txt', 'w', encoding="utf-8") as filehandle:
    for listitem in March:
        filehandle.write('%s\n' % listitem)

In [9]:
len(March)

93

In [17]:
len(March[9])

47